In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# SPOTIFY MÜZİK ANALİZİ

![](https://i2.cnnturk.com/i/cnnturk/75/650x325/5e27ea222af1071fac9be5ae.jpg)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn import datasets, linear_model
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
import seaborn as sns             # görselleştirme yapmak için kullanacağız.
import missingno                  # eksik verileri daha iyi okumak için kullanacağız.
from sklearn import preprocessing   # ön işleme aşamasında label encoding vb. için dahil ettik.
import re    

In [ ]:
df = pd.read_csv("../input/spotifyclassification/data.csv").copy()

In [ ]:
df.head(10) 

In [ ]:
df.shape #2017x17 lik bir veri tabanımız var öyleyse bu veri tabanımızı biraz inceleyelim

In [ ]:
df.dtypes

NOT: burada unnamed olarak şarkıların idsi ifade edilmiş, bunun dışında burada şarkıların özellikleri oluşturulmuş artık neye göre yaptıysa arkadaşlar neyse bu bizim işimiz değil ve bunun dışında müziğin ismi ve müzisyenin adına da yer verilmiş toplamda 17 değişken tipi belirlenmiş şimdi bunlar üzerinde daha fazla bilgi edinelim ve analizde bulunalım :)

In [ ]:
df.info()

In [ ]:
df.isnull().sum() # ohh miss gibi hiçbir nan veri yok itina ile yapılmış :D

In [ ]:
df.describe().T #tüm sayısal verilerin istatistiksel verilerini görüyoruz

In [ ]:
corr = df.corr()
corr

NOT: burada 17x17 lik bir korelasyon matrisi çıktı he bu böyle çirkin ve karmaşık duruyo biz bunu ısı haritasını çıkaralım.

In [ ]:
sns.heatmap(corr, xticklabels = corr.columns.values, yticklabels = corr.columns.values);

In [ ]:
df.columns

In [ ]:
df.columns = ['MuzisyenID','acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence', 'target',
       'song_title', 'artist']
df.head()

NOT: şimdi burada aslında kolon isimlerini türkçeleştirebiliz ama ben bir iki tanesinin çevisinin mantığını bulamadım o yüzden şimdilik yapmayacağım ama basit şeyler var akustiklik müzik süresi gibisinden enerji gibi vs. ve ayrıca burada her kolonun fazla sayıda uniqe değeri olacaktır çünkü her müzik kendi içinde özel değerlere sahiptir bu yüzden ben artist yani müziyen üzerinden işlem yapacağım.

In [ ]:
df.artist.nunique() # maşallah 1343 tane de sanatçı varmış :D

In [ ]:
df.artist

In [ ]:
df.sort_values('energy', axis = 0, ascending = False).head(10)[["energy","artist","song_title",]]

NOT: enerjisi en yüksek 10 tane şarkıyı enerji değerlerini müzisyenini ve müzik başlığını getirdim dinleyelim bakalım o kadar enerjik mi :D

bir de en az enerjik parçalarımızdan 10 tanesine bakalım

In [ ]:
df.sort_values('energy', axis = 0, ascending = False).tail(10)[["energy","artist","song_title",]]

In [ ]:
df[(df['artist'] == "Eminem")].sort_values('energy', axis = 0, ascending=False)

Eminem'in 2 tane şarkısı varmış sağlık olsun bunları da enerjilerine göre sıraladım tabi bunu süreye vs. vs. sıralayabilirdim ben böyle yapmak istedim.

In [ ]:
sns.scatterplot(x = "danceability", y = "tempo", data = df); # tempo ile danceability arasında böyle bir ilişki görüyoruz

In [ ]:
chosen = ["energy", "liveness", "tempo", "valence", "loudness", "speechiness", "acousticness", "danceability", "instrumentalness"]
text1 = df["artist"] + " - " + df["song_title"]
text2 = text1.values

# X = data_frame.drop(droppable, axis=1).values
X = df[chosen].values
y = df["danceability"].values

min_max_scaler = MinMaxScaler()
X = min_max_scaler.fit_transform(X)

pca = PCA(n_components=3)
pca.fit(X)

X = pca.transform(X)

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

trace = go.Scatter3d(
    x=X[:,0],
    y=X[:,1],
    z=X[:,2],
    text=text2,
    mode="markers",
    marker=dict(
        size=8,
        color=y
    )
)

fig = go.Figure(data=[trace])
py.iplot(fig, filename="test-graph")